# Postgres database

A Postgres database and a front-end (using Adminer) was created using the following `docker-compose.yml` file:

```yaml
version: "3"

services:
  db:
    image: postgres
    container_name: db
    restart: always
    environment:
      - POSTGRES_PASSWORD=password
    ports:
      - 5432:5432

  adminer:
    image: adminer
    container_name: adminer
    restart: always
    depends_on:
      - db    
    ports: 
      - 8080:8080

```

In [73]:
import psycopg2

In [74]:
def database_connection(host, port, username, password, database=None):
    """Returns a database connection."""
    
    # Preconditions
    assert isinstance(host, str), f"Expected a str, got {type(host)}"
    assert len(host) > 0
    assert isinstance(port, int), f"Expected an int, got {type(port)}"
    assert port > 0
    assert isinstance(username, str), f"Expected a str, got {type(username)}"
    assert len(username) > 0
    assert isinstance(password, str), f"Expected a str, got {type(password)}"

    if database is None:
        conn = psycopg2.connect(
            database="postgres", host=host, port=str(port), user=username, password=password
        )
    else:
        conn = psycopg2.connect(
            database=database, host=host, port=str(port), user=username, password=password
        )        
    
    conn.autocommit = True
    
    return conn

## Create the database

In [75]:
conn = database_connection("192.168.99.100", 5432, "postgres", "password")

conn.autocommit = True
cursor = conn.cursor()
sql = """CREATE database comet"""

try:
    cursor.execute(sql)
except:
    print(f"Can't make the database as it already exists!")

conn.close()

## Create a table

In [81]:
# Mapping from a Python type to a database type
type_mappings = {
    int: "BIGINT",
    float: "DOUBLE PRECISION",
    str: "TEXT",
    bool: "BOOLEAN"
}

In [82]:
schema = {
    "name": str,
    "age": int,
    "employed": bool,
    "height": float
}

In [83]:
name_to_type = [f"{name} {type_mappings[tpe]}" for name, tpe in schema.items()]
name_to_type

['name TEXT', 'age BIGINT', 'employed BOOLEAN', 'height DOUBLE PRECISION']

In [84]:
table_name = "person"
field_spec = ", ".join(name_to_type)
stmt = f"CREATE TABLE {table_name} ({field_spec})"
stmt

'CREATE TABLE person (name TEXT, age BIGINT, employed BOOLEAN, height DOUBLE PRECISION)'

In [85]:
conn = database_connection("192.168.99.100", 5432, "postgres", "password", "comet")
cursor = conn.cursor()
cursor.execute(stmt)
conn.close()

comet


## Check if a table exists

In [86]:
query = """
SELECT tablename
FROM pg_catalog.pg_tables
WHERE schemaname != 'pg_catalog' AND 
    schemaname != 'information_schema' AND tablename like '%person%'; 
"""

In [87]:
conn = database_connection("192.168.99.100", 5432, "postgres", "password", "comet")
cursor = conn.cursor()
cursor.execute(query)
conn.close()

comet


In [88]:
result = cursor.fetchone()

In [89]:
result

('person',)

## Insert data into a table

In [ ]:
query = """
INSERT INTO person ('name', 'age', 'employed', 'height') VALUES ('Bob', 23, true, 1.68)
"""

In [ ]:
conn = database_connection("192.168.99.100", 5432, "postgres", "password", "comet")
cursor = conn.cursor()
cursor.execute(query)
conn.close()